In [1]:
import pandas as pd
import time
import requests
import urllib.parse

In [2]:
def add_lat_long(dataframe,
                 address_column: str):
    
    print(f"attempting to add lat and long to {dataframe.shape[0]} rows")
    start = time.time()
    
    # add lat and long as dataframe columns
    dataframe["latitude"] = None
    dataframe["longitude"] = None
    
    # add lat and long
    for idx, details in dataframe.iterrows():

        address = details[address_column]
        
        url = 'https://nominatim.openstreetmap.org/search/' + urllib.parse.quote(str(address)) +'?format=json'
        response = requests.get(url).json()

        try:
            dataframe.loc[idx, "latitude"] = float(response[0]["lat"])
            dataframe.loc[idx, "longitude"] = float(response[0]["lon"])
            
        except:
            print(f"lat, long failed for {address}")

    end = time.time()
    print(f"completed lat, long function in {round(end-start, 2)} seconds")
    return dataframe

In [3]:
df = pd.DataFrame(data = {"college" : ["Pace University",
                                       "Baruch College",
                                       "Hunter College"],
                         "location" : ["Pace University Midtown Center, New York, NY 10176",
                                       "17 Lexington Ave, New York, NY 10010",
                                       "413 E 69th St, New York, NY 10021"]})

df

,college,location
0,Pace University,"Pace University Midtown Center, New York, NY 1..."
1,Baruch College,"17 Lexington Ave, New York, NY 10010"
2,Hunter College,"413 E 69th St, New York, NY 10021"


In [4]:
df = add_lat_long(dataframe = df,
                  address_column = "location")

attempting to add lat and long to 3 rows
lat, long failed for Pace University Midtown Center, New York, NY 10176
completed lat, long function in 2.75 seconds


In [5]:
df

,college,location,latitude,longitude
0,Pace University,"Pace University Midtown Center, New York, NY 1...",None,None
1,Baruch College,"17 Lexington Ave, New York, NY 10010",40.739142,-73.984845
2,Hunter College,"413 E 69th St, New York, NY 10021",40.765562,-73.956111


In [6]:
from yelp.client import Client

# find your API key at: https://fusion.yelp.com/ then click "Manage API Key"
#MY_API_KEY = "your key here"
MY_API_KEY = ""
client = Client(MY_API_KEY)

In [7]:
url = 'https://api.yelp.com/v3/businesses/search'

term = 'bars'
location = '55 Lexington Ave, New York, NY 10010'
SEARCH_LIMIT = 25
radius = 1600*3 # 3 miles

In [8]:
url_params = {
                'term': term.replace(' ', '+'),
                'location': location.replace(' ', '+'),
                'limit': SEARCH_LIMIT,
                'radius': radius
            }

headers = {"Authorization": f"Bearer {MY_API_KEY}"}

In [9]:
import requests

# make the request to Yelp, get a response
response = requests.get(url, headers=headers, params=url_params)
print(response)
print(type(response.text))

<Response [200]>
<class 'str'>


In [10]:
import pandas as pd

# turn your Yelp json response into a pandas dataframe
df = pd.DataFrame.from_dict(response.json()['businesses'])
print(f"shape of your dataframe: {df.shape}")

shape of your dataframe: (25, 16)


In [11]:
# subset the columns in your dataframe
df = df[["name", "review_count",
         "rating", "price",
         "distance", "location"]]

df.head(3)

,name,review_count,rating,price,distance,location
0,Chapel Bar,30,4.0,NaN,295.039409,"{'address1': '281 Park Ave S', 'address2': '',..."
1,Jungly,8,4.0,NaN,2821.888561,"{'address1': '12-23 Jackson Ave', 'address2': ..."
2,Crown Alley,132,5.0,$$,1404.233086,"{'address1': '263 West 19th St', 'address2': N..."


In [12]:
# split your location column into multiple columns
df["street"] = [d.get('address1') for d in df['location']]
df["city"] = [d.get('city') for d in df['location']]
df["state"] = [d.get('state') for d in df['location']]
df["country"] = [d.get('country') for d in df['location']]


# drop the location column
df = df.drop(['location'], axis=1)

In [13]:
df["address"] = df['street'] + ', ' + df['city'] + ', ' + df['state']

In [14]:
bars_lat_long_df = add_lat_long(dataframe = df, address_column = "address")

attempting to add lat and long to 25 rows
lat, long failed for 45 W 45th St between 5th & 6th Ave, New York, NY
lat, long failed for 20 W 23rd St Cellar, New York, NY
lat, long failed for 66 Park Ave E 38th St, New York, NY
completed lat, long function in 26.03 seconds


In [15]:
bars_lat_long_df

,name,review_count,rating,price,distance,street,city,state,country,address,latitude,longitude
0,Chapel Bar,30,4.0,NaN,295.039409,281 Park Ave S,New York,NY,US,"281 Park Ave S, New York, NY",40.73927,-73.986775
1,Jungly,8,4.0,NaN,2821.888561,12-23 Jackson Ave,Queens,NY,US,"12-23 Jackson Ave, Queens, NY",40.743991,-73.950193
2,Crown Alley,132,5.0,$$,1404.233086,263 West 19th St,New York,NY,US,"263 West 19th St, New York, NY",40.742787,-73.999686
3,Ampersand,246,4.5,$$,179.440249,294 3rd Ave,New York,NY,US,"294 3rd Ave, New York, NY",40.676978,-73.986626
4,Nothing Really Matters,63,4.5,$$,2356.039867,1627 Broadway,New York,NY,US,"1627 Broadway, New York, NY",40.761398,-73.984317
5,Valerie,482,4.5,$$,1804.479815,45 W 45th St between 5th & 6th Ave,New York,NY,US,"45 W 45th St between 5th & 6th Ave, New York, NY",None,None
6,Thyme Bar,211,4.0,$$$,618.616186,20 W 23rd St Cellar,New York,NY,US,"20 W 23rd St Cellar, New York, NY",None,None
7,The Green Room,28,4.0,$$,137.387515,156 E 23rd St,New York,NY,US,"156 E 23rd St, New York, NY",40.738863,-73.983874
8,Patent Pending,290,4.0,$$$,801.615550,49 W 27th St,New York,NY,US,"49 W 27th St, New York, NY",40.745219,-73.990241
9,Raines Law Room,1550,4.5,$$$,953.473098,48 W 17th St,New York,NY,US,"48 W 17th St, New York, NY",40.73873,-73.994593


In [16]:
bars_lat_long_df.to_excel("bars.xlsx")